# How to FineTune Pretrained Models with   Trainer 



In [ ]:
!python -m pip install --upgrade pip
!pip3 install torch==2.2.1 torchvision torchaudio   xformers --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft \
    accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps  trl peft accelerate bitsandbytes
!pip install datasets
!pip install hyperopt
!pip install optuna    
pass

In [7]:
!python -m xformers.info
!python -m bitsandbytes
!nvidia-smi

Unable to find python bindings at /usr/local/dcgm/bindings/python3. No data will be captured.
xFormers 0.0.25
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.decoderF:               available
memory_efficient_attention.flshattF@v2.5.6:        available
memory_efficient_attention.flshattB@v2.5.6:        available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:

#  How Fine Tune a Model by  Using Trainer Class

In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
# Model and tokenizer selection
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # 2 for positive/negative sentiment
# Load dataset (smaller subsets) - adjust these ranges as needed
train_data = load_dataset("imdb", split="train").shuffle(seed=42).select(range(1000))
test_data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(100))
# Define a function to preprocess data

def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length")

# Preprocess data
train_data = train_data.map(preprocess_function, batched=True)
test_data = test_data.map(preprocess_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    save_steps=100,
    save_total_limit=2,
    num_train_epochs=2,
    evaluation_strategy="epoch"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Train the model
trainer.train()
# Evaluate the model
predictions = trainer.predict(test_data)
print(f"Test Loss: {predictions.metrics['test_loss']}")
# Save the trained model
trainer.save_model("bert-classification-trained")
tokenizer.save_pretrained("bert-classification-trained")


import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# Load the fine-tuned model and tokenizer (replace with your model name)
model_name = "bert-classification-trained"  # Replace with the actual model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Function to preprocess text for inference
def preprocess_text(text):
    """Preprocesses a single text sample for inference."""
    encoding = tokenizer(text, truncation=True, padding="max_length", return_tensors="pt")
    return encoding
def predict(text):
    # Preprocess the sample text
    input_ids = preprocess_text(text)
    # Perform inference using the model
    with torch.no_grad():
        outputs = model(**input_ids)
        logits = outputs.logits.squeeze(0)  # Remove batch dimension if present
    # Get the predicted class (positive or negative sentiment) based on the highest logit
    predicted_class = torch.argmax(logits).item()
    predicted_label = "positive" if predicted_class == 1 else "negative"

    print(f"Predicted sentiment for '{text}': {predicted_label}")
    
    
# Sample text for inference (replace with your text)
text = "This movie was absolutely fantastic! I highly recommend it."  # Replace with the text you want to classify
predict(text)

text = "This movie was not good I dislike."  # Replace with the text you want to classify
predict(text)    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)  # Convert NumPy array to Tensor
    predictions = torch.argmax(logits_tensor, dim=-1)
    accuracy = (predictions == torch.from_numpy(labels)).to(torch.float).mean()
    return {"accuracy": accuracy}

## Step 1 - Dataset Setup
In ordering to perform fine tuning of a pretrained model to perform Question Answering , we first requiere have a dataset with the information tailored a specic case in particular a dataset that contains a context, question and answer.

In [125]:

from datasets import load_dataset, load_metric
from datasets import DatasetDict, Dataset

datasets = load_dataset("squad")
# Use dictionary comprehension to create train and test subsets
train_dataset = datasets["train"].select(range(100))
test_dataset = datasets["train"].select(range(100, 100 + 10))

# Now you have two separate datasets for training and testing:
# train_dataset: Contains the first 100 samples
# test_dataset: Contains the next 10 samples

# (Optional) Update the original dataset dictionary for clarity
datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [89]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [31]:
from datasets import load_dataset, load_metric
from datasets import DatasetDict, Dataset

datasets = load_dataset("squad")
# Use dictionary comprehension to create train and test subsets
train_dataset = datasets["train"].select(range(100))
test_dataset = datasets["train"].select(range(100, 100 + 10))


# Now you have two separate datasets for training and testing:
# train_dataset: Contains the first 100 samples
# test_dataset: Contains the next 10 samples

# (Optional) Update the original dataset dictionary for clarity
datasets = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [114]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 100
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10
    })
})

In [115]:
datasets["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [116]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=1):

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [124]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5733b7f74776f41900661131,University_of_Notre_Dame,"The university is affiliated with the Congregation of Holy Cross (Latin: Congregatio a Sancta Cruce, abbreviated postnominals: ""CSC""). While religious affiliation is not a criterion for admission, more than 93% of students identify as Christian, with over 80% of the total being Catholic. Collectively, Catholic Mass is celebrated over 100 times per week on campus, and a large campus ministry program provides for the faith needs of the community. There are multitudes of religious statues and artwork around campus, most prominent of which are the statue of Mary on the Main Building, the Notre Dame Grotto, and the Word of Life mural on Hesburgh Library depicting Christ as a teacher. Additionally, every classroom displays a crucifix. There are many religious clubs (catholic and non-Catholic) at the school, including Council #1477 of the Knights of Columbus (KOC), Baptist Collegiate Ministry (BCM), Jewish Club, Muslim Student Association, Orthodox Christian Fellowship, The Mormon Club, and many more. The Notre Dame KofC are known for being the first collegiate council of KofC, operating a charitable concession stand during every home football game and owning their own building on campus which can be used as a cigar lounge. Fifty-seven chapels are located throughout the campus.",What amount of the student body of Notre Dame identifies as Catholic?,"{'text': ['over 80%'], 'answer_start': [251]}"


In [117]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import DatasetDict, Dataset
import torch
from transformers import default_data_collator
# Model and tokenizer selection
model_name = "distilbert-base-uncased"  # Replace with a compatible model (e.g., BERT, RoBERTa, etc.)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)



Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [132]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        #print(answers)

     
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [133]:
pad_on_right = tokenizer.padding_side == "right"

In [134]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [135]:
features = prepare_train_features(datasets['train'][:1])


In [136]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [137]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10
    })
})

In [138]:
# Training arguments
training_args = TrainingArguments(
  output_dir=f"{model_name}-finetuned",
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=2,  # Adjust based on GPU memory
  per_device_eval_batch_size=2,
  num_train_epochs=3,
  weight_decay=0.01,
)
from transformers import default_data_collator
data_collator = default_data_collator

# Create a Trainer
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["test"],
  data_collator=data_collator,
  tokenizer=tokenizer,
)


trainer.train()
trainer.save_model("test-squad-trained")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,5.443951
2,No log,4.982282
3,No log,4.830709


In [18]:
trainer.save_model("test-squad-trained")

In [19]:
trainer

# Inference

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Replace with the actual name of your fine-tuned question answering model
model_name = "test-squad-trained"

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model.to(device)
# Prepare the question and context (replace with your actual context)
question = "What amount of the student body of Notre Dame identifies as Catholic?"
context = """The university is affiliated with the Congregation of Holy Cross (Latin: Congregatio a Sancta Cruce, abbreviated postnominals: "CSC"). While religious affiliation is not a criterion for admission, more than 93% of students identify as Christian, with over 80% of the total being Catholic. Collectively, Catholic Mass is celebrated over 100 times per week on campus, and a large campus ministry program provides for the faith needs of the community.""" 
# Tokenize the question and context
encoding = tokenizer(question, context, return_tensors="pt", padding="max_length", truncation=True)
encoding.to(device)
# Perform inference using the finetuned model
with torch.no_grad():
    outputs = model(**encoding)
    # Move outputs to CPU if necessary
    start_logits = outputs.start_logits.to("cpu")
    end_logits = outputs.end_logits.to("cpu")
    # Extract answer span from start and end logits
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits) + 1
    # Handle empty answer case
    if answer_start == answer_end:
        final_answer = "No answer found by the model."
    else:
        # Get the answer tokens and convert them back to text
        answer = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0][answer_start:answer_end])
        final_answer = tokenizer.convert_tokens_to_string(answer)
print(f"Answer: {final_answer}")


Answer: 80 %


In [ ]:
#Cpu

In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Replace with the actual name of your fine-tuned question answering model
model_name = "test-squad-trained"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Prepare the question and context (replace with your actual context)
question = "What amount of the student body of Notre Dame identifies as Catholic?"
context = """The university is affiliated with the Congregation of Holy Cross (Latin: Congregatio a Sancta Cruce, abbreviated postnominals: "CSC"). While religious affiliation is not a criterion for admission, more than 93% of students identify as Christian, with over 80% of the total being Catholic. Collectively, Catholic Mass is celebrated over 100 times per week on campus, and a large campus ministry program provides for the faith needs of the community.""" 

# Tokenize the question and context
encoding = tokenizer(question, context, return_tensors="pt", padding="max_length", truncation=True)

# Move input tensors to appropriate device (CPU)
encoding = {key: tensor.to("cpu") for key, tensor in encoding.items()}

# Perform inference using the finetuned model
with torch.no_grad():
    outputs = model(**encoding)

    # Move outputs to CPU if necessary
    start_logits = outputs.start_logits.to("cpu")
    end_logits = outputs.end_logits.to("cpu")

    # Extract answer span from start and end logits
    answer_start = torch.argmax(start_logits)
    answer_end = torch.argmax(end_logits) + 1

    # Handle empty answer case
    if answer_start == answer_end:
        final_answer = "No answer found by the model."
    else:
        # Get the answer tokens and convert them back to text
        answer = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0][answer_start:answer_end])
        final_answer = tokenizer.convert_tokens_to_string(answer)

print(f"Answer: {final_answer}")


Answer: 80 %


## Evaluation
Evaluating our model will require a bit more work, as we will need to map the predictions of our model back to parts of the context. The model itself predicts logits for the start and en position of our answers: if we take a batch from our validation datalaoder, here is the output our model gives us:

In [42]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [45]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [46]:
validation_features = datasets["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["test"].column_names
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [47]:
raw_predictions = trainer.predict(validation_features)


In [51]:
import collections
import numpy as np


In [53]:
squad_v2=False

In [54]:
final_predictions = postprocess_qa_predictions(datasets["test"], validation_features, raw_predictions.predictions)


Post-processing 10 example predictions split into 10 features.


  0%|          | 0/10 [00:00<?, ?it/s]

In [55]:
final_predictions

OrderedDict([('573387acd058e614000b5cb5', '1918'),
             ('573387acd058e614000b5cb3', '1918'),
             ('573387acd058e614000b5cb4', '1918'),
             ('573388ce4776f41900660cc3', '1920s'),
             ('573388ce4776f41900660cc4', '1920s'),
             ('573388ce4776f41900660cc7', '1920s'),
             ('573388ce4776f41900660cc5', '1920s'),
             ('573388ce4776f41900660cc6', '1920s'),
             ('57338a51d058e614000b5cf0', '1883'),
             ('57338a51d058e614000b5cf1', '1933')])

In [56]:
metric = load_metric("squad")


/tmp/wsuser/ipykernel_131/1715067326.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/datasets/load.py:759: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [57]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 0.0, 'f1': 0.0}

In [ ]:
#datasets = load_dataset("ruslanmv/ai-medical-chatbot")
# Rename columns
#train_dataset = train_dataset.rename_columns({"Description": "question", "Patient": "context", "Doctor": "answers"})
#test_dataset = test_dataset.rename_columns({"Description": "question", "Patient": "context", "Doctor": "answers"})
# Create a unique id column
#train_dataset= train_dataset.add_column("id", [i for i in range(len(train_dataset))])
#test_dataset = test_dataset.add_column("id", [i for i in range(len(test_dataset))])


### Multiple GPU Running on Jypyter Notebook

If we are interested to use  `torch.multiprocessing` to execute a Python function in parallel across multiple processes. We can run the following code:

```python
import torch.multiprocessing as mp
def foo(process_idx):
    print(process_idx)
if __name__ == "__main__":
    mp.spawn(foo, nprocs=2)  # should print 0 1
    
```    
It crashes in Jupyter environment and works fine as a regular python program.

```
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'foo' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'foo' on <module '__main__' (built-in)>

ProcessExitedException: process 0 terminated with exit code 1

```

However if we run by using this the code can run 

In [4]:
import torch.multiprocessing as mp
def foo(process_idx):
    print(process_idx)
if __name__ == "__main__":
    mp.start_processes(foo, nprocs=2, start_method="fork")  # should print 0 1

0
1


But the previos trick does not work with   Pytorch Lightning, so currently we have troubles to run code on multiple GPUs with Pytorch Lightning inside a Jupyter notebook. So the simplest workaround shoud do not use jupyter notebook. 

In [5]:
import os

code_string = '''
import torch.multiprocessing as mp
def foo(process_idx):
    print(process_idx)
if __name__ == "__main__":
    mp.spawn(foo, nprocs=2)  # should print 0 1
'''
with open("launch.py", "w") as f:
    f.write(code_string)

os.system("python launch.py")


0
1


0

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# Model and tokenizer selection
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Dataset loading
dataset = load_dataset("glue", "mnli", split="train").select(range(100))
text_field = "premise"  

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    num_train_epochs=2,
)

# Trainer initialization
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    dataset_text_field=text_field,
    compute_metrics=None,
)

print("Trainer for single-GPU loaded")

# Start training
trainer.train()


/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BartForCausalLM were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246:

Trainer for single-GPU loaded


/opt/conda/envs/Python-RT23.1-CUDA/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


TrainOutput(global_step=50, training_loss=9.102232666015626, metrics={'train_runtime': 6.8474, 'train_samples_per_second': 29.208, 'train_steps_per_second': 7.302, 'total_flos': 3747087949824.0, 'train_loss': 9.102232666015626, 'epoch': 2.0})

Now with two gpus

In [ ]:
import os
code_string = '''
import torch
import torch.distributed as dist
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer  # Assuming SFTTrainer is a custom training class
from transformers import TrainingArguments
import os

# Model and tokenizer selection
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Dataset loading
dataset = load_dataset("glue", "mnli", split="train").select(range(100))
text_field = "premise"

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,  # Adjust batch size based on GPU memory
    save_steps=100,
    save_total_limit=2,
    num_train_epochs=2
)


# Distributed training setup using `torch.distributed`
def init_distributed(local_rank):
    # Address provided by environment variables (if available)
    master_addr = os.getenv('MASTER_ADDR', default='localhost')
    master_port = os.getenv('MASTER_PORT', default=12345)

    # Initialize distributed process group
    dist.init_process_group(backend='nccl', init_method=f'tcp://{master_addr}:{master_port}', rank=local_rank, world_size=torch.cuda.device_count())
    
def cleanup_distributed():
    dist.destroy_process_group()


# Define the main function with distributed setup
def main(local_rank):
    init_distributed(local_rank)

    if local_rank != 0:
        # Disable printing on non-master process to reduce clutter
        torch.distributed.barrier()

    # Move model and optimizer to the corresponding GPU
    model.to('cuda:' + str(local_rank))

    # Trainer initialization (assuming SFTTrainer supports distributed training)
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        dataset_text_field=text_field,
        compute_metrics=None,
    )

    if local_rank == 0:
        print("Trainer for multi-GPU loaded")

    # Start training with distributed configuration
    trainer.train()

    cleanup_distributed()


if __name__ == "__main__":
    world_size = torch.cuda.device_count()

    # Launch multiple processes, each handling a GPU
    torch.multiprocessing.spawn(main, args=(), nprocs=world_size)
'''
with open("launch.py", "w") as f:
    f.write(code_string)

os.system("NCCL_DEBUG=INFO python launch.py")


In [ ]:
import os
code_string = '''
import torch
import torch.distributed as dist
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer  # Assuming SFTTrainer is a custom training class
from transformers import TrainingArguments
import os

# Model and tokenizer selection
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Dataset loading
dataset = load_dataset("glue", "mnli", split="train").select(range(100))
text_field = "premise"

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # Adjust batch size based on GPU memory
    save_steps=100,
    save_total_limit=2,
    num_train_epochs=2
)

def init_distributed(local_rank):
    # Address provided by environment variables (if available)
    master_addr = os.getenv('MASTER_ADDR', default='localhost')
    master_port = os.getenv('MASTER_PORT', default=12345)

    # Initialize distributed process group
    dist.init_process_group(
        backend='nccl',
        init_method=f'tcp://{master_addr}:{master_port}',
        rank=local_rank,
        world_size=torch.cuda.device_count()
    )

def cleanup_distributed():
    dist.destroy_process_group()

# Define the main function with distributed setup
def main(local_rank):
    init_distributed(local_rank)

    # Move model and optimizer to the corresponding GPU
    model.to(f'cuda:{local_rank}')

    # Trainer initialization (assuming SFTTrainer supports distributed training)
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        dataset_text_field=text_field,
        compute_metrics=None,
    )

    # Start training with distributed configuration
    trainer.train()

    cleanup_distributed()

if __name__ == "__main__":
    # Launch multiple processes, each handling a GPU
    torch.multiprocessing.spawn(main, args=(), nprocs=torch.cuda.device_count())

'''
with open("launch.py", "w") as f:
    f.write(code_string)

os.system("NCCL_DEBUG=INFO python launch.py")


In [ ]:
import torch
import torch.distributed as dist
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os

# Model and tokenizer selection
model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Dataset loading
dataset = load_dataset("glue", "mnli", split="train").select(range(100))
text_field = "premise"

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    num_train_epochs=2,
)
# Distributed training setup (assuming all GPUs are available on a single machine)
def init_distributed(rank):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12345"
    dist.init_process_group(backend='nccl', world_size=torch.cuda.device_count(),    rank=rank)
def cleanup_distributed():
    dist.destroy_process_group()

# Define the main function
def main(rank):
    init_distributed(rank)

    # Trainer initialization
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        dataset_text_field=text_field,
        compute_metrics=None,
    )

    print("Trainer for multi-GPU loaded")

    # Start training
    trainer.train()

    cleanup_distributed()

if __name__ == "__main__":
    import multiprocessing
    multiprocessing.set_start_method('spawn')  # Set start method to 'spawn'

    world_size = torch.cuda.device_count()
    processes = []
    for rank in range(world_size):
        p = torch.multiprocessing.Process(target=main, args=(rank,))
        processes.append(p)
        p.start()
